In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread(r"C:\Users\vrinda\Desktop\2nd sem\CV\dtest.png")

## scaling

In [10]:
#  scale factor
scale_factor = .75

In [11]:
# Scaling the image
scaled_img = cv2.resize(img, (int(scale_factor*img.shape[1]),int(scale_factor*img.shape[0])), interpolation=cv2.INTER_LINEAR)
cv2.imshow('Org',img)
cv2.imshow('Scaled',scaled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

In [13]:
# Detect the  keypoints and computing descriptors
key_1, des1 = sift.detectAndCompute(img, None)
key_2, des2 = sift.detectAndCompute(scaled_img, None)

In [14]:
# Initialize BFMatcher with default params
bf = cv2.BFMatcher()

In [15]:
# Match descriptors
matches = bf.knnMatch(des1, des2, k=2)

In [16]:
# Apply ratio test
good = []
for m, n in matches:
    if m.distance < 0.50 * n.distance:
        good.append(m)


In [17]:
# Drawing  matches
final_img = cv2.drawMatches(img, key_1, scaled_img, key_2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [18]:

cv2.imshow('Matching Result', final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


## rotaion

In [19]:
#  rotation angle in degrees
angle = 30

In [20]:
# Define rotation matrix
rotation_matrix = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), angle, 1)

In [21]:
# Apply rotation to image
rotated= cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))

In [22]:
# Extract SIFT keypoints and descriptors from original and rotated images
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(img, None)
keypoints2, descriptors2 = sift.detectAndCompute(rotated, None)


In [23]:
# Display original and rotated images with keypoints
img_with_keypoints1 = cv2.drawKeypoints(img, keypoints1, None)
img_with_keypoints2 = cv2.drawKeypoints(rotated, keypoints2, None)
cv2.imshow("Original Image with Keypoints", img_with_keypoints1)
cv2.imshow("Rotated Image with Keypoints", img_with_keypoints2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## affine


In [24]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

# Find keypoints and descriptors
key, d = sift.detectAndCompute(img, None)


In [25]:
# Define the affine transformation matrix
theta = np.radians(50)
scale = 1.0
tx, ty = 100, -50

M = np.array([
    [scale * np.cos(theta), -scale * np.sin(theta), tx],
    [scale * np.sin(theta), scale * np.cos(theta), ty]
])

In [26]:
# Apply the affine transformation
img_affine = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

In [27]:
# Find keypoints and descriptors in the transformed image
kp_affine, des_affine = sift.detectAndCompute(img_affine, None)

In [28]:
# Draw keypoints on the images
img_kp = cv2.drawKeypoints(img, key, None)
img_affine_sift = cv2.drawKeypoints(img_affine, kp_affine, None)

In [29]:
# displaying the images
cv2.imshow('Original', img_kp)
cv2.imshow('Affine Transformation', img_affine_sift)
cv2.waitKey(0)
cv2.destroyAllWindows()


## perspective

In [42]:
# Define the four corners of the original image
pts1 = np.float32([[0, 260], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1], [img.shape[1]-1, 0]])


In [43]:
# Define the four corners of the desired output image
pts2 = np.float32([[0, 0], [0, 400], [400, 400], [400, 0]])

In [44]:
# Calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(pts1, pts2)

In [45]:
# Apply the perspective transform to the original image
perspective_img = cv2.warpPerspective(img, M, (400, 400))
# cv2.imshow('Perspective',perspective_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [54]:
# Extract SIFT keypoints and descriptors from original and transformed images
sift = cv2.SIFT_create()
keypoints1, descriptors1 = sift.detectAndCompute(img, None)
keypoints2, descriptors2 = sift.detectAndCompute(perspective_img, None)

In [55]:
# Use a brute-force matcher to find matches between descriptors
bf = cv2.BFMatcher()
matches = bf.match(descriptors1, descriptors2)

In [57]:
# Draw matching lines on a new image
# Drawing  matches
final_img = cv2.drawMatches(img, keypoints1, perspective_img, keypoints2, matches[:20], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [58]:
# Display image with matching lines
cv2.imshow("Matches", final_img)
cv2.waitKey(0)
cv2.destroyAllWindows()